In [46]:
import pickle
import numpy as np
import pandas as pd

In [25]:
%store -r test

# Take the name of the different classes (e.g. '0', '1', '2', ...)
classes = test.groupby(['label'])['label'].unique().index.to_list()

In [49]:
classes

[0, 1, 2, 3, 5, 6, 7, 8, 9]

In [78]:
predicted_test = test.copy()

models = []
for n_class in classes: # Can be thrown an expection beacuse this is the labels of the test set and not of the models (train set)
    model = pickle.load(open('models/'+str(n_class)+'hmm_model.pkl', 'rb'))
    models.append(model)

log_likelihoods = pd.DataFrame()
log_likelihoods['predicted_class'] = classes

for i in range(len(test)):

    true_label = test['label'].iloc[i]
    list_log_likelihoods = []
    for model in models:
        
        angle = test['angles'].iloc[i]
        X = [[one] for one in angle]
        list_log_likelihoods.append(model.score(X))
        pass

    log_likelihoods['log_likelihood'] = list_log_likelihoods
    log_likelihoods['true_class'] = true_label

    max_log_likelihood = log_likelihoods.loc[log_likelihoods['log_likelihood'].idxmax()]
    predicted_test['log_likelihood'] = max_log_likelihood['log_likelihood']
    predicted_test['predicted_class'] = np.nan
    if max_log_likelihood['predicted_class'] == max_log_likelihood['true_class']:
        predicted_test.loc[i, 'predicted_class'] = 1
    else:
        predicted_test.loc[i, 'predicted_class'] = 0

predicted_test



# model_0.score_samples(angles_0)

,label,x,y,speed,strokes,angles,log_likelihood,predicted_class
382,0,"[513.0, 469.0, 429.0, 408.0, 388.0, 373.0, 364...","[105.0, 139.0, 187.0, 226.0, 268.0, 314.0, 360...",1.715,1,"[2.48, 2.27, 2.06, 2.02, 1.89, 1.76, 1.56, 1.3...",-21.745743,NaN
892,6,"[936.0, 931.0, 907.0, 831.0, 809.0, 791.0, 771...","[12.0, 8.0, 11.0, 30.0, 38.0, 46.0, 60.0, 76.0...",0.476,1,"[-2.47, 3.02, 2.9, 2.79, 2.72, 2.53, 2.47, 2.3...",-21.745743,NaN
123,6,"[786.0, 760.0, 725.0, 709.0, 650.0, 610.0, 571...","[154.0, 152.0, 156.0, 159.0, 175.0, 191.0, 211...",1.279,1,"[-3.06, 3.03, 2.96, 2.88, 2.76, 2.67, 2.44, 2....",-21.745743,NaN
499,0,"[690.0, 653.0, 622.0, 586.0, 546.0, 506.0, 486...","[106.0, 116.0, 130.0, 149.0, 174.0, 205.0, 222...",1.279,1,"[2.88, 2.72, 2.66, 2.58, 2.48, 2.44, 2.29, 2.1...",-21.745743,NaN
212,6,"[613.0, 602.0, 588.0, 570.0, 531.0, 518.0, 478...","[230.0, 231.0, 240.0, 255.0, 289.0, 302.0, 344...",1.560,1,"[3.05, 2.57, 2.45, 2.42, 2.36, 2.33, 2.28, 2.2...",-21.745743,NaN
...,...,...,...,...,...,...,...,...
122,NaN,NaN,NaN,NaN,NaN,NaN,-21.745743,NaN
124,NaN,NaN,NaN,NaN,NaN,NaN,-21.745743,NaN
125,NaN,NaN,NaN,NaN,NaN,NaN,-21.745743,NaN
126,NaN,NaN,NaN,NaN,NaN,NaN,-21.745743,NaN
